# Concept Space Traversal

Concept space traversal is the exploration of high-dimensional embedding spaces where data points represent semantic relationships between concepts. It enables navigation and manipulation of semantic relationships captured by embeddings, providing a powerful tool for exploring abstract concept spaces in various AI and machine learning applications.

## Key Components

1. **Embedding Space**: 
   - High-dimensional vector space
   - Similar concepts positioned closer together

2. **Similarity Measures**:
   - Typically cosine similarity or Euclidean distance
   - Closer points indicate higher semantic similarity

3. **Traversal Methods**:
   a. Linear Interpolation:
      - Create intermediate points between concepts
      - Use weighted averages of vector representations

   b. Vector Arithmetic:
      - Perform operations (e.g., addition, subtraction) on vectors
      - Explore relationships and analogies

   c. Nearest Neighbor Search:
      - Find closest points to a given vector
      - Discover related concepts

## Applications

- Concept generation
- Similarity search
- Analogy discovery
- Semantic relationship exploration


In [ ]:
import os

import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz

In [ ]:
import fiftyone.utils.huggingface as fouh

color_swap = fouh.load_from_hub(
    "Voxel51/ColorSwap",
    name="colorswap_full",
    max_samples=551,
    persistent=True,
    overwrite=True,
    )

The Concept Traversal Plugin for FiftyOne allows users to navigate the space of concepts in their dataset using both text and images. 

Key points:

- You select a starting image from their dataset, then iteratively add text concepts with relative strengths to move around the multimodal embedding space.

- Behind the scenes, it generates embedding vectors for the text prompts, combines them with the starting image vector, and performs a similarity search on the dataset.

- Creating the plugin required generating a multimodal similarity index (e.g. using a CLIP model) on the dataset first.

To use the plugin, a similarity index that supports prompts (i.e., embeds both text and images) must be present on the dataset. 

This can be created using the `fiftyone.brain` module, specifically the `compute_similarity` function, which takes the dataset, a `brain_key`, the `model_name` (e.g., `clip-vit-base32-torch`), and the `metric` (e.g., `cosine`) as arguments.

The plugin can be installed by running the command `fiftyone plugins download https://github.com/jacobmarks/concept-interpolation`.

The plugin provides two main operators:

1. `open_interpolation_panel`: Opens the interpolation panel when clicked, but is only activated when the dataset has a similarity index.

2. `interpolator`: Runs the actual interpolation between the two text prompts.

In summary, this FiftyOne plugin enables users to explore the latent space between two text concepts by interpolating between their embeddings and visualizing the results, providing an interactive way to understand the relationships between different text prompts.


In [ ]:
clip_model = foz.load_zoo_model(
    name="clip-vit-base32-torch",
    install_requirements=True,
)

color_swap.compute_embeddings(
    model=clip_model,
    embeddings_field="clip_embeddings",
    progress=True,  
)

In [ ]:
import os 

sim_index = fob.compute_similarity(
    color_swap,
    brain_key="concept_embeddings",
    embeddings="clip_embeddings",
    model="clip-vit-base32-torch",
    metric="cosine",
    )

In [ ]:
fob.compute_visualization(
    color_swap,
    embeddings="clip_embeddings",
    method="umap",
    brain_key = "umap_2d_clip",
    num_dims=2,
    progress=True, 
)

In [ ]:
session = fo.launch_app(color_swap)